## Setup: Imports and Installations

following the tutorial from :https://docs.argilla.io/en/latest/guides/llms/examples/train-reward-model-rlhf.html

# Part 1 With noraml weight

In [ ]:
%%capture
!pip install transformers
!pip install datasets
!pip install pandas
!pip install torch
!pip install importlib

#preprocessing
!pip install trl peft
!pip install -q -U git+https://github.com/lvwerra/trl.git git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/accelerate.git git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops
!pip install sentencepiece

In [ ]:
import json
import os
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from datasets import Dataset, DatasetDict
from google.colab import drive
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_ as clip_grad_norm
from torch.utils.data import DataLoader

from tqdm import tqdm
from transformers import (AdamW, AutoModel, AutoModelForSequenceClassification, AutoConfig,
                          AutoTokenizer, Trainer, TrainingArguments, get_polynomial_decay_schedule_with_warmup,
                          get_linear_schedule_with_warmup)
from transformers import DistilBertModel, DistilBertTokenizer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset
from trl import RewardTrainer
from peft import LoraConfig, TaskType, prepare_model_for_kbit_training, PeftModel, PromptLearningConfig,MODEL_TYPE_TO_PEFT_MODEL_MAPPING,get_peft_model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from google.colab import drive

drive.mount("/content/MyDrive")



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable%: {100 * trainable_params / all_param}"
    )

# 1 GPT2

In [ ]:
torch.cuda.empty_cache()
torch.cuda.mem_get_info()

(42044293120, 42481549312)

In [ ]:
with open('training_dataset.json',encoding="cp437", errors='ignore') as json_file:
    train = json.load(json_file)

max_tokens = 512

full = pd.DataFrame(train)

full = full[(full['length_0']<=max_tokens)]

train, test = train_test_split(full, test_size=0.1, random_state=18625541)

train = pd.DataFrame(train)
train = train[(train['length_0']<=max_tokens)&(train['length_1']<=max_tokens)]
list_of_dicts = train.to_dict(orient='records')
train = Dataset.from_list(list_of_dicts)

test = pd.DataFrame(test)
test = test[(test['length_0']<=max_tokens)&(test['length_1']<=max_tokens)]
list_of_dicts = test.to_dict(orient='records')
test = Dataset.from_list(list_of_dicts)


In [ ]:
model_name = "gpt2-medium"


model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": max_tokens, "return_tensors": "pt"}

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = examples["chat_1"]
    prompt_plus_rejected_response = examples["chat_0"]

    # Then tokenize these modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

test = test.map(formatting_func)
train = train.map(formatting_func)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/303 [00:00<?, ? examples/s]

Map:   0%|          | 0/2716 [00:00<?, ? examples/s]

In [ ]:
print_trainable_parameters(model)
training_args = TrainingArguments(
    output_dir="./reward_model",
    per_device_train_batch_size=4,
    evaluation_strategy="steps",
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=300,
    logging_steps=150,
    num_train_epochs=4,

)
trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=test,
    max_length=max_tokens
)

trainer.train()

trainable params: 354,824,192 || all params: 354,824,192 || trainable%: 100.0


/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:136: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strate

Step,Training Loss,Validation Loss,Accuracy
150,0.811200,0.459826,0.801980
300,0.424900,0.482120,0.858086
450,0.517100,0.457892,0.805281
600,0.472600,0.530502,0.825083
750,0.432400,0.516286,0.828383
900,0.382500,0.601951,0.851485
1050,0.339500,0.369525,0.861386
1200,0.412700,0.592931,0.877888
1350,0.330800,0.623947,0.877888
1500,0.202200,0.569762,0.891089


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr

TrainOutput(global_step=2716, training_loss=0.2916414544798313, metrics={'train_runtime': 2184.1674, 'train_samples_per_second': 4.974, 'train_steps_per_second': 1.243, 'total_flos': 0.0, 'train_loss': 0.2916414544798313, 'epoch': 4.0})

In [ ]:
# Flatten the list of batched outputs
with open('eval_dataset.json',encoding="cp437", errors='ignore') as json_file:
    eval = json.load(json_file)

max_tokens = 512

eval = pd.DataFrame(eval)
list_of_dicts = eval.to_dict(orient='records')
eval = Dataset.from_list(list_of_dicts)

eval_encoded = eval.map(lambda examples: tokenizer(examples['chat'], truncation=True, padding='longest',max_length=max_tokens), batched=True)

print_trainable_parameters(model)

eval_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask'])

dataloader = DataLoader(eval_encoded, batch_size=1,num_workers=0)

# Disable gradient calculation
outputs = []
model.eval()
with torch.no_grad():
    for batch in tqdm(dataloader):
        torch.cuda.empty_cache()

        # Move batch to device
        batch = {k: v.to('cuda') for k, v in batch.items()}
        # Get model's output
        output = model(**batch)
        logits = output.logits.detach().cpu().numpy()
        outputs.append(logits)
flattened_outputs = np.concatenate([batch[0] for batch in outputs], axis=0)

print(len(flattened_outputs))
flattened_outputs

13586


array([ 1.7503318 ,  9.521019  , -1.9541194 , ..., -7.261831  ,
       -0.56273746, -2.6336484 ], dtype=float32)

In [ ]:
eval_df = pd.DataFrame(list_of_dicts)
eval_df['predictions'] = flattened_outputs.tolist()
eval_df.to_json("GPT2.json", orient="records", indent=4)

# 2 ELECTRA

In [ ]:
torch.cuda.empty_cache()
torch.cuda.mem_get_info()

(31451578368, 42481549312)

In [ ]:
with open('trainin_dataset.json',encoding="cp437", errors='ignore') as json_file:
    train = json.load(json_file)

max_tokens = 512

full = pd.DataFrame(train)

full = full[(full['length_0']<=max_tokens)]

train, test = train_test_split(full, test_size=0.1, random_state=18625541)

train = pd.DataFrame(train)
train = train[(train['length_0']<=max_tokens)&(train['length_1']<=max_tokens)]
list_of_dicts = train.to_dict(orient='records')
train = Dataset.from_list(list_of_dicts)

test = pd.DataFrame(test)
test = test[(test['length_0']<=max_tokens)&(test['length_1']<=max_tokens)]
list_of_dicts = test.to_dict(orient='records')
test = Dataset.from_list(list_of_dicts)


In [ ]:
model_name = "OpenAssistant/reward-model-electra-large-discriminator"


model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'eos_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": max_tokens, "return_tensors": "pt"}

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = examples["chat_1"]
    prompt_plus_rejected_response = examples["chat_0"]

    # Then tokenize these modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

test = test.map(formatting_func)
train = train.map(formatting_func)

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

Map:   0%|          | 0/1970 [00:00<?, ? examples/s]

In [ ]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 8,392,706 || all params: 342,484,994 || trainable%: 2.4505324750082336


In [ ]:
training_args = TrainingArguments(
    output_dir="./reward_model_electra",
    per_device_train_batch_size=4,
    evaluation_strategy="steps",
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    logging_steps=200,
    num_train_epochs=4,
)
trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=test,
    max_length=max_tokens
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:136: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation str

Step,Training Loss,Validation Loss,Accuracy
200,0.604000,0.597138,0.710843
400,0.598100,0.541728,0.738956
600,0.504100,0.507564,0.748996
800,0.511300,0.480599,0.761044
1000,0.485200,0.447678,0.769076
1200,0.408600,0.474170,0.775100
1400,0.498000,0.445216,0.771084
1600,0.373100,0.449525,0.785141
1800,0.415200,0.447581,0.783133


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


TrainOutput(global_step=1972, training_loss=0.4828656295250202, metrics={'train_runtime': 1183.6072, 'train_samples_per_second': 6.658, 'train_steps_per_second': 1.666, 'total_flos': 0.0, 'train_loss': 0.4828656295250202, 'epoch': 4.0})

# 3 ROBERTA

In [ ]:
torch.cuda.empty_cache()
torch.cuda.mem_get_info()

(42044293120, 42481549312)

In [ ]:
with open('trainin_dataset.json',encoding="cp437", errors='ignore') as json_file:
    train = json.load(json_file)

max_tokens = 512

full = pd.DataFrame(train)

full = full[(full['length_0']<=max_tokens)]

train, test = train_test_split(full, test_size=0.1, random_state=18625541)

train = pd.DataFrame(train)
train = train[(train['length_0']<=max_tokens)&(train['length_1']<=max_tokens)]
list_of_dicts = train.to_dict(orient='records')
train = Dataset.from_list(list_of_dicts)

test = pd.DataFrame(test)
test = test[(test['length_0']<=max_tokens)&(test['length_1']<=max_tokens)]
list_of_dicts = test.to_dict(orient='records')
test = Dataset.from_list(list_of_dicts)


In [ ]:

model_name = "argilla/roberta-base-reward-model-falcon-dolly"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'eos_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": max_tokens, "return_tensors": "pt"}

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = examples["chat_1"]
    prompt_plus_rejected_response = examples["chat_0"]

    # Then tokenize these modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

test = test.map(formatting_func)
train = train.map(formatting_func)

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

Map:   0%|          | 0/1970 [00:00<?, ? examples/s]

In [ ]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 2,362,370 || all params: 83,890,178 || trainable%: 2.8160269251067747


In [ ]:
training_args = TrainingArguments(
    output_dir="./reward_model_roberta",
    per_device_train_batch_size=1,
    evaluation_strategy="steps",
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    logging_steps=200,
    num_train_epochs=4,
)
trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=test,
    peft_config=peft_config,
    max_length=max_tokens
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:136: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation str

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 21>:21                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1539 in train                    │
│                                                                                                  │
│   1536 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1537 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1538 │   │   )                                                                                 │
│ ❱ 1539 │   │   return inner_training_loop(                                                       │
│   1540 │   │   │   args=args,                                                                    │
│   1541 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1542 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1801 in _inner_training_loop     │
│                                                                                                  │
│   1798 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1799 │   │   │   │                                                                             │
│   1800 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1801 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1802 │   │   │   │                                                                             │
│   1803 │   │   │   │   if (                                                                      │
│   1804 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2636 in training_step            │
│                                                                                                  │
│   2633 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2634 │   │                                                                                     │
│   2635 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2636 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2637 │   │                                                                                     │
│   2638 │   │   if self.args.n_gpu > 1:                                                           │
│   2639 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:169 in compute_loss        │
│                                                                                                  │
│   166 │   │   │   raise NotImplementedError(                                                     │
│   167 │   │   │   │   "compute_loss is only implemented for RewardDataCollatorWithPadding, ple   │
│   168 │   │   │   )                                                                              │
│ ❱ 169 │   │   rewards_chosen = model(input_ids=inputs["input_ids_chosen"], attention_mask=inpu   │
│   170 │   │   rewards_rejected = model(                    

# 4 OPT

In [ ]:
torch.cuda.empty_cache()
torch.cuda.mem_get_info()

(36140810240, 42481549312)

In [ ]:
with open('trainin_dataset.json',encoding="cp437", errors='ignore') as json_file:
    train = json.load(json_file)

max_tokens = 512

full = pd.DataFrame(train)

full = full[(full['length_0']<=max_tokens)]

train, test = train_test_split(full, test_size=0.1, random_state=18625541)

train = pd.DataFrame(train)
train = train[(train['length_0']<=max_tokens)&(train['length_1']<=max_tokens)]
list_of_dicts = train.to_dict(orient='records')
train = Dataset.from_list(list_of_dicts)

test = pd.DataFrame(test)
test = test[(test['length_0']<=max_tokens)&(test['length_1']<=max_tokens)]
list_of_dicts = test.to_dict(orient='records')
test = Dataset.from_list(list_of_dicts)


In [ ]:
model_name = "AdamG012/chat-opt-350m-reward-deepspeed"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": max_tokens, "return_tensors": "pt"}

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = examples["chat_1"]
    prompt_plus_rejected_response = examples["chat_0"]

    # Then tokenize these modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

test = test.map(formatting_func)
train = train.map(formatting_func)

Some weights of the model checkpoint at AdamG012/chat-opt-350m-reward-deepspeed were not used when initializing OPTForSequenceClassification: ['rwtranrsformer.decoder.layers.11.self_attn.out_proj.weight', 'rwtranrsformer.decoder.layers.8.fc1.bias', 'rwtranrsformer.decoder.layers.3.fc2.bias', 'rwtranrsformer.decoder.layers.16.self_attn_layer_norm.weight', 'rwtranrsformer.decoder.layers.5.self_attn.v_proj.bias', 'rwtranrsformer.decoder.layers.12.fc1.bias', 'rwtranrsformer.decoder.layers.15.self_attn.q_proj.weight', 'rwtranrsformer.decoder.layers.2.self_attn.q_proj.weight', 'rwtranrsformer.decoder.layers.8.self_attn.k_proj.weight', 'rwtranrsformer.decoder.layers.0.final_layer_norm.bias', 'rwtranrsformer.decoder.embed_positions.weight', 'rwtranrsformer.decoder.layers.22.self_attn.v_proj.weight', 'rwtranrsformer.decoder.layers.23.self_attn.q_proj.weight', 'rwtranrsformer.decoder.layers.14.fc1.weight', 'rwtranrsformer.decoder.layers.10.self_attn.q_proj.weight', 'v_head.weight', 'rwtranrsform

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Map:   0%|          | 0/469 [00:00<?, ? examples/s]

In [1]:
training_args = TrainingArguments(
    output_dir="./reward_model_opt",
    per_device_train_batch_size=1,
    evaluation_strategy="steps",
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=10,
    max_steps=1,
    logging_steps=1,
)
trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=test,
    max_length=max_tokens,
)

trainer.train()

NameError: ignored

# 5 DEBERTA

In [ ]:
torch.cuda.empty_cache()
torch.cuda.mem_get_info()

(22740008960, 42481549312)

In [ ]:
with open('training_dataset.json',encoding="cp437", errors='ignore') as json_file:
    train = json.load(json_file)

max_tokens = 512

full = pd.DataFrame(train)

full = full[(full['length_0']<=max_tokens)]

train, test = train_test_split(full, test_size=0.1, random_state=18625541)

train = pd.DataFrame(train)
train = train[(train['length_0']<=max_tokens)&(train['length_1']<=max_tokens)]
list_of_dicts = train.to_dict(orient='records')
train = Dataset.from_list(list_of_dicts)

test = pd.DataFrame(test)
test = test[(test['length_0']<=max_tokens)&(test['length_1']<=max_tokens)]
list_of_dicts = test.to_dict(orient='records')
test = Dataset.from_list(list_of_dicts)


In [ ]:
len(train), len(test)

(2716, 303)

In [ ]:
model_name = "OpenAssistant/reward-model-deberta-v3-large-v2"
model_nmae = "reward_model_debert_augmented/checkpoint-500"


model1 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model1.config.pad_token_id = model1.config.eos_token_id

def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": max_tokens, "return_tensors": "pt"}

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = examples["chat_1"]
    prompt_plus_rejected_response = examples["chat_0"]

    # Then tokenize these modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

test = test.map(formatting_func)
train = train.map(formatting_func)


Map:   0%|          | 0/303 [00:00<?, ? examples/s]

Map:   0%|          | 0/2716 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./reward_model_debert_augmented",
    per_device_train_batch_size=4,
    evaluation_strategy="steps",
    learning_rate=2e-6,
    weight_decay=0.01,
    warmup_steps=300,
    logging_steps=150,
    num_train_epochs=2,
)
trainer = RewardTrainer(
    model=model1,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=test,
    max_length=max_tokens
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:136: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation s

Step,Training Loss,Validation Loss,Accuracy
150,0.538800,0.481489,0.749175
300,0.481800,0.449655,0.792079
450,0.481100,0.409695,0.815182
600,0.415800,0.397066,0.825083
750,0.375400,0.382414,0.834983
900,0.376500,0.382072,0.841584
1050,0.397000,0.373335,0.851485
1200,0.349400,0.364485,0.848185
1350,0.283100,0.368568,0.851485


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


TrainOutput(global_step=1358, training_loss=0.4096897020606826, metrics={'train_runtime': 1382.3841, 'train_samples_per_second': 3.929, 'train_steps_per_second': 0.982, 'total_flos': 0.0, 'train_loss': 0.4096897020606826, 'epoch': 2.0})

In [ ]:
with open('eval_dataset.json',encoding="cp437", errors='ignore') as json_file:
    eval = json.load(json_file)

max_tokens = 512

eval = pd.DataFrame(eval)
list_of_dicts = eval.to_dict(orient='records')
eval = Dataset.from_list(list_of_dicts)


In [ ]:

eval_encoded = eval.map(lambda examples: tokenizer(examples['chat'], truncation=True, padding='longest',max_length=max_tokens), batched=True)


eval_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask'])

dataloader = DataLoader(eval_encoded, batch_size=4)


Map:   0%|          | 0/13586 [00:00<?, ? examples/s]

In [ ]:

# Disable gradient calculation
outputs = []
with torch.no_grad():
    for batch in tqdm(dataloader):
        # Move batch to device
        batch = {k: v.to('cuda') for k, v in batch.items()}
        # Get model's output
        output = model1(**batch)
        outputs.append(output)
# Flatten the list of batched outputs
flattened_outputs = np.concatenate([batch.logits.cpu().numpy() for batch in outputs], axis=0)
print(len(flattened_outputs))

100%|██████████| 3397/3397 [07:49<00:00,  7.23it/s]


13586


In [ ]:
eval_df = pd.DataFrame(list_of_dicts)
eval_df['predictions'] = flattened_outputs
eval_df.to_json("results_deberta.json", orient="records", indent=4)